# Создание вопросно-ответного приложения на основе данных SQL

Работа LLM со структурированными данными и неструктурированным текстом может качественно отличаться.
Если при работе с неструктурированным текстом обычно генерируется текст, который можно найти с помощью поиска по векторному хранилищу, то для взаимодействия со структурированными данными LLM зачастую нужно писать запросы на подходящем DSL (domain specific language), например, SQL. 

Раздел содержит пример разработки вопросно-ответных приложений, которые используют табличные данные в базе данных.
Здесь представлены два варианта реализации: с использованием цепочек и агентов.
Оба варианта позволяют задавать вопрос о данных, которые хранятся в базе, и возвращает ответ на естественном языке.
При этом основное отличие решения, которое использует агента, в том, что агент может выполнять запросы к базе данных в цикле столько раз, сколько нужно для ответа на вопрос.

:::danger

Вопросно-ответные приложения на базе данных SQL требуют выполнения SQL запросов, которые сгенерировала модель.
Это связано с определенными рисками.
Убедитесь, что вашим цепочкам/агентам предоставлены только необходимые разрешения.
Это снизит, но не устранит полностью, риски создания системы, управляемой моделью.
Подробнее — в разделе [Безопасность](/docs/security).

:::

## Архитектура

В общем случае работу таких приложений можно представить этапами:

1. Преобразование вопроса в запрос DSL. Модель преобразует ввод пользователя в SQL запрос.
2. Выполнение SQL запроса.
3. Ответ на вопрос. Модель отвечает на ввод пользователя, используя результаты запроса.

:::note

Запрос данных из CSV-файлов может следовать аналогичным этапам.

Подробнее в разделе [How to do question answering over CSVs](/docs/how_to/sql_csv).

:::

## Подготовка к работе

Установите необходимые пакеты и задайте переменные окружения:

In [2]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community langchain-openai faiss-cpu

При работе с этим руководством используется модель OpenAI.

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Раскомментируйте строки ниже, чтобы использовать LangSmith. Не обязательно.
# if not os.environ.get("LANGCHAIN_API_KEY"):
#    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
#    os.environ["LANGCHAIN_TRACING_V2"] = "true"

Ниже в примере используется SQLite-соединение с базой данных Chinook.

Для работы с блокнотом следуйте [инструкции по установке](https://database.guide/2-sample-databases-sqlite/), чтобы создать `Chinook.db` в той же папке, что и этот блокнот:

1. Сохраните файл [`Chinook_Sqlite.sql`](https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql) как `Chinook.sql`
2. Запустите `sqlite3 Chinook.db`
3. Выполните `.read Chinook.sql`
4. Проверьте `SELECT * FROM Artist LIMIT 10;`

После создания базы данных `Chinhook.db` в той же папке, что и блокнот, вы можете обращаться к ней с помощью класс `SQLDatabase`, который работает на основе SQLAlchemy:

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

После создания базы данных попробуйте подключить ее к LLM.

## Цепочки {#chains}

Цепочки (соединения [Runnable-объектов](/docs/concepts#langchain-expression-language) GigaChain) поддерживают приложения с предсказуемыми этапами работы.

Вы можете создать простую цепочку, которая принимает вопрос и выполняет действия:

- преобразует вопрос в SQL-запрос;
- выполняет запрос;
- использует результат для ответа на исходный вопрос.

Но существуют сценарии, которые не получится реализовать с помощью этой схемы.
Например, такое приложение выполнит SQL-запрос для любого ввода пользователя, даже для простого приветствия "привет".
При этом, некоторые вопросы требуют более одного запроса для получения ответа.
Обработка таких ситуаций рассмотрена в подразделе об [Агенты](#agents).

### Преобразование вопроса в SQL-запрос

Первый шаг в SQL-цепочке или агенте — это преобразовать ввод пользователя в SQL-запрос.
Для этого в GigaChain есть встроенная цепочка: [create_sql_query_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.sql_database.query.create_sql_query_chain.html).

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [3]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM "Employee"\nLIMIT 1;'

Выполните запрос, чтобы убедиться, что он работает:

In [4]:
db.run(response)

'[(8,)]'

Вы можете исследовать полный промпт следующим образом:

<!--
Мы можем посмотреть на [трассировку LangSmith](https://smith.langchain.com/public/c8fa52ea-be46-4829-bde2-52894970b830/r), чтобы лучше понять, что делает эта цепочка. Мы также можем непосредственно исследовать цепочку для её промптов. Посмотрев на промпт (ниже), мы можем увидеть, что он:

* Специфичен для диалекта. В этом случае он явно ссылается на SQLite.
* Имеет определения для всех доступных таблиц.
* Имеет три примера строк для каждой таблицы.

Эта техника вдохновлена статьями, такими как [эта](https://arxiv.org/pdf/2204.00498.pdf), которые предполагают, что показ примеров строк и явное указание таблиц улучшает производительность. Мы также можем исследовать полный промпт следующим образом:
-->

In [5]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### Выполнение SQL-запроса

Выполнение запроса — самая опасная часть создания SQL-цепочки.
Взвесьте все за и против перед выполнением автоматических запросов к своим данным.
Уменьшите количество прав при доступе к базе данных до минимума.
Подумайте стоит ли добавить в свои цепочки этап, на котором человек самостоятельно принимает решение выполнять запрос или нет.
Как это сделать, смотрите ниже.

Для добавления запросов в цепочку используйте `QuerySQLDatabaseTool`:

In [6]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

### Ответ на вопрос

Чтобы создать итоговый ответ достаточно объединить исходный вопрос и результат SQL-запроса.

Для этого передайте вопрос и результат в LLM:

In [7]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many employees are there"})

'There are a total of 8 employees.'

Рассмотрим, что происходит в примере выше.
Предположим, что выполняется вызов представленной цепочки.

- После срабатывания первого `RunnablePassthrough.assign`, вы получаете Runnable-объект с двумя элементами: `{"question": question, "query": write_query.invoke(question)}`  
  Где `write_query` сгенерирует SQL-запрос для ответа на вопрос.
- После срабатывания второго `RunnablePassthrough.assign` добавляется третий элемент `"result"`, который содержит `execute_query.invoke(query)`, где `query` был вычислен на предыдущем шаге.
- Эти три входных значения преобразуются в промпт и передаются в LLM.
- `StrOutputParser()` извлекает строковое содержимое из выходного сообщения.

В примере LLM, инструменты, промпты и другие цепочки объединяются вместе, но поскольку каждый из этих элементов реализует интерфейс `Runnable`, их входные и выходные данные могут быть связаны между собой.

### Смотрите также

Для более сложной генерации запросов вы можете создать промпты с несколькими примерами или добавить шаги проверки запросов.
Подробнее об этих методиках в разделах:

- [Стратегии формирования промптов](/docs/how_to/sql_prompting).
- [Проверка запросов](/docs/how_to/sql_query_checking).
- [Большие базы данных](/docs/how_to/sql_large_db).

## Агенты {#agents}

В GigaChain есть встроенный SQL-агент, который предоставляет более гибкий способ работы с базами данных SQL, чем цепочка.
Основные преимущества использования SQL-агента:

- способность отвечать на вопросы, как на основе схемы, так и на основе содержимого базы данных. Например, описывая конкретную таблицу.
- возможность восстановиться после ошибок. Агент выполняет запрос, обрабатывает трассировку ошибки и заново генерирует исправленный запрос.
- способность выполнять запросы к базе данных столько раз, сколько необходимо для ответа на вопрос пользователя.
- экономия токенов, обусловленная запросом только схем из соответствующих таблиц.

Для инициализации агента используйте `SQLDatabaseToolkit`.
С помощью этого класса можно:

* создавать и выполнять запросы;
* проверять синтаксис запросов;
* получать описания таблиц и выполнять другие действия.

In [8]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x113403b50>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x113403b50>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x113403b50>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

### Системный промпт

Вам также нужно создать системный промпт, который включает инструкций о том, как агент должен себя вести.

In [32]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

### Инициализация агента

Сначала установите необходимый пакет GigaGraph.

In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet langgraph

В примерах ниже для разработки агента используется предварительно подготовленный агент [GigaGraph](/docs/concepts/#langgraph):

In [33]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

Посмотрите, как агент отвечает на следующий вопрос:

In [34]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Which country's customers spent the most?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vnHKe3oul1xbpX0Vrb2vsamZ', 'function': {'arguments': '{"query":"SELECT c.Country, SUM(i.Total) AS Total_Spent FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId GROUP BY c.Country ORDER BY Total_Spent DESC LIMIT 1"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 557, 'total_tokens': 610}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-da250593-06b5-414c-a9d9-3fc77036dd9c-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': 'SELECT c.Country, SUM(i.Total) AS Total_Spent FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId GROUP BY c.Country ORDER BY Total_Spent DESC LIMIT 1'}, 'id': 'call_vnHKe3oul1xbpX0Vrb2vsamZ'}])]}}
----
{'action': {'messages': [ToolMessage(content='Error: (sqlite3.OperationalError) no s

Агент выполняет несколько запросов, пока не получит нужную информацию:
1. Перечисляет доступные таблицы.
2. Извлекает схему для трех таблиц.
3. Выполняет запросы к нескольким таблицам с использованием операции объединения.

После этого агент может использовать результат последнего запроса для формирования ответа на исходный вопрос.

Похожим образом агент может обрабатывать качественные вопросы:

In [35]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Describe the playlisttrack table")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WN0N3mm8WFvPXYlK9P7KvIEr', 'function': {'arguments': '{"table_names":"playlisttrack"}', 'name': 'sql_db_schema'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 554, 'total_tokens': 571}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-be278326-4115-4c67-91a0-6dc97e7bffa4-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'playlisttrack'}, 'id': 'call_WN0N3mm8WFvPXYlK9P7KvIEr'}])]}}
----
{'action': {'messages': [ToolMessage(content="Error: table_names {'playlisttrack'} not found in database", name='sql_db_schema', id='fe32b3d3-a40f-4802-a6b8-87a2453af8c2', tool_call_id='call_WN0N3mm8WFvPXYlK9P7KvIEr')]}}
----
{'agent': {'messages': [AIMessage(content='I apologize for the error. Let me first check the available tables in the database.', additional_kw

### Работа со столбцами с высокой кардинальностью

Для фильтрации столбцов, которые содержат имена собственные, такие как адреса, названия песен или имена артистов, сначала нужно дважды проверить орфографию, чтобы корректно отфильтровать данные.

Вы можете добиться этого, создав векторное хранилище со всеми уникальными именами собственными, которые существуют в базе данных.
После этого агент сможет выполнять запросы к созданному хранилищу каждый раз, когда пользователь включает имя собственное в свой вопрос, чтобы найти правильное написание этого слова.
Таким образом, агент сможет убедиться, что он понимает, о каком объекте говорит пользователь, прежде чем создавать целевой запрос.

Сначала вам нужно получить уникальные значения для каждого необходимого объекта.
Для этого определите функцию, которая будет разбирать результат в список элементов:

In [36]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")
albums[:5]

['Big Ones',
 'Cidade Negra - Hits',
 'In Step',
 'Use Your Illusion I',
 'Voodoo Lounge']

Используя эту функцию, вы можете создать *инструмент ретривера*, который агент сможет выполнять по своему усмотрению.

In [39]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vector_db = FAISS.from_texts(artists + albums, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

Вызов инструмента.

In [40]:
print(retriever_tool.invoke("Alice Chains"))

Alice In Chains

Alanis Morissette

Pearl Jam

Pearl Jam

Audioslave


Таким образом, если агент решит, что ему нужно создать фильтр на основе имени исполнителя, например "Alice Chains", он сначала сможет использовать инструмент ретривера для получения соответствующих значений столбца.

Итоговое решение будет выглядеть так:

In [50]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, messages_modifier=system_message)

In [51]:
for s in agent.stream(
    {"messages": [HumanMessage(content="How many albums does alis in chain have?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_r5UlSwHKQcWDHx6LrttnqE56', 'function': {'arguments': '{"query":"SELECT COUNT(*) AS album_count FROM Album WHERE ArtistId IN (SELECT ArtistId FROM Artist WHERE Name = \'Alice In Chains\')"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 612, 'total_tokens': 652}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-548353fd-b06c-45bf-beab-46f81eb434df-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': "SELECT COUNT(*) AS album_count FROM Album WHERE ArtistId IN (SELECT ArtistId FROM Artist WHERE Name = 'Alice In Chains')"}, 'id': 'call_r5UlSwHKQcWDHx6LrttnqE56'}])]}}
----
{'action': {'messages': [ToolMessage(content='[(1,)]', name='sql_db_query', id='093058a9-f013-4be1-8e7a-ed839b0c90cd', tool_call_id='call_r5UlSwHKQcWDHx6LrttnqE56')]}

В примере выше агент использовал инструмент `search_proper_nouns` для того, чтобы проверить, как правильно выполнить запрос к базе данных для этого конкретного исполнителя.